In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os 

# File to Load (Remember to Change These)
file_to_load = os.path.join('purchase_data.csv')

# Read Purchasing File and store into Pandas data frame
df = pd.read_csv(file_to_load)
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


Player Count:
Display the total number of players

In [2]:
#What the difference is between len(df) and len(df['SN'].unique())? Are there repeated records?
df_total_players = len(df["SN"].unique())
df_total = pd.DataFrame({'Total Players': [df_total_players]})
df_total

,Total Players
0,576


Purchasing Analysis (Total)
Run basic calculations to obtain number of unique items, average price, etc.
Create a summary data frame to hold the results
Optional: give the displayed data cleaner formatting
Display the summary data frame

In [3]:
#Uses the original dataframe

#Total of Unique Items
unique_items = df['Item ID'].nunique()

#Average Price Per Item
avg_price = df['Price'].mean()

#Total or Purchases Made
total_purchases = len(df)

#Total Revenue of All Purchases
total_revenue = df['Price'].sum()

#New dataframe containing the calculated values
summary_df = pd.DataFrame({'Total Unique Items': [unique_items],
                           'Average Price': [avg_price],
                           'Total Purchase': [total_purchases],
                           'Total Revenue': [total_revenue]})

#Prettified version
summary_df_styled = summary_df.style.format({'Average Price': "${:,.2f}", 'Total Revenue': '${:,.2f}'})
summary_df_styled

,Total Unique Items,Average Price,Total Purchase,Total Revenue
0,183,$3.05,780,"$2,379.77"


In [4]:
new_df = df.drop_duplicates(subset=['SN'], keep='first')

Gender Demographics
Percentage and Count of Male Players
Percentage and Count of Female Players
Percentage and Count of Other / Non-Disclosed

In [5]:
#Based off of active player gender demographics.

#Female
total_female = len(new_df[(new_df['Gender'] == 'Female')])
percent_female = round((total_female / df_total_players) * 100, 2)

#Male
total_male = len(new_df[(new_df['Gender'] == 'Male')])
percent_male = round((total_male / df_total_players) * 100, 2)

#Unassigned Gender
total_unassigned = len(new_df[(new_df['Gender'] == 'Other / Non-Disclosed')]) 
percent_unassigned = round((total_unassigned / df_total_players) * 100, 2)

#Gender Demographics DataFrame
rows = ["Female", "Male", "Other / Non-Disclosed"]
gender_demo_df = pd.DataFrame({'Total Counts': [total_female, total_male, total_unassigned], 
                               'Percentage of Players': [percent_female, percent_male, percent_unassigned]}, index=[rows])

gender_demo_df

,Total Counts,Percentage of Players
Female,81,14.06
Male,484,84.03
Other / Non-Disclosed,11,1.91


In [35]:
#groupby on original dataframe instead of trying to make a new one. Manipulate the dataframe we already have
#Incomplete: missing average spend per head

#New gender demographics df
gender_demo_df = df[['Gender', 'SN', 'Price']]

#number of purchases made by each gender
gender_demo_totals = gender_demo_df['Gender'].value_counts()

#average amount spent per purchase by gender
gender_group_by_price = gender_demo_df.groupby('Gender').agg('mean')
gender_group_by_s = gender_group_by_price['Price']

#sum total purchase amounts by gender
gender_group_by_total = gender_demo_df.groupby('Gender').agg('sum')
gender_group_avg_sum_s = gender_group_by_total['Price']

#average spend per head by gender
gender_group_by_head = gender_demo_df.groupby('Gender')
spend_by_gender = gender_group_by_head['Price'].sum()
unique_purchasers = gender_group_by_head['SN'].nunique()
avg_spend_per_head = spend_by_gender / unique_purchasers


#Final Gender Demographics Summary
#Incomplete
gender_demographics = pd.DataFrame({"Total Purchases": gender_demo_totals, 
                                    "Average Purchase Price": gender_group_by_s, 
                                    "Total Purchase Amount": gender_group_avg_sum_s,
                                    "Average Purchase Amount Per Person": avg_spend_per_head})

gender_demographics_clean = gender_demographics.style.format({'Average Purchase Price': "${:,.2f}", 
                                                              'Total Purchase Amount': '${:,.2f}',
                                                              'Average Purchase Amount Per Person': '${:,.2f}'})

gender_demographics_clean

,Total Purchases,Average Purchase Price,Total Purchase Amount,Average Purchase Amount Per Person
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


Age Demographics
Establish bins for ages
Categorize the existing players using the age bins. Hint: use pd.cut()
Calculate the numbers and percentages by age group
Create a summary data frame to hold the results
Optional: round the percentage column to two decimal points
Display Age Demographics Table

In [43]:
#New dataframe of players
age_df = df.drop_duplicates(subset=['SN'], keep='first')

#Bins and labels
bins = [0, 10, 15, 20, 25, 30, 35, 40, 45]
groupings = ['>10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

#Add the new column to the age_df
age_df['Age Range'] = pd.cut(age_df['Age'], bins, right=False, labels=groupings)

#Group the dataframe by the new Age Range column.
age_group = age_df.groupby('Age Range')

#Total Number of Players Per Age Range
total_age_per_range = age_group['SN'].count()

#Percentage of Players Per Age Range
average_age_per_range = (total_age_per_range / len(age_df)) * 100

#Create the dataframe containing the new series
age_demographics = pd.DataFrame({"Total Count": total_age_per_range, "Percentage of Players": average_age_per_range})

#Prettified version
age_demographics_styled = age_demographics.round(2)
age_demographics_styled

/Users/ashleymcgee/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Total Count,Percentage of Players
Age Range,,
>10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,11,1.91


In [34]:
#Purchase Data By Age Grouping
#Values for age ranges seem a little off. Skews the rest of the data.

df_copy = df

#Bins and Labels
bins = [0, 10, 15, 20, 25, 30, 35, 40, 45]
groupings = ['>10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

#New Age Range column for the original df
df_copy['Age Range'] = pd.cut(df_copy['Age'], bins, right=False, labels=groupings)

#Group the original df by Age Range
purchase_age = df_copy.groupby('Age Range')

#Total Number of Purchases Made By Age Range
total_purchases_per_range = purchase_age['Item Name'].count()

#Average Purchase Price By Age Range
avg_purchase_price_range = purchase_age['Price'].mean()

#Total Purchase Value By Age Range
total_purchase_value_range = purchase_age['Price'].sum()

#Average Spend Per Head By Age Range
unique_players_by_range = purchase_age['SN'].nunique()
avg_spend_per_age_range = total_purchase_value_range / unique_players_by_range


#Dataframe summary for purchases made by age range.
purchase_age_demo = pd.DataFrame({"Purchase Count": total_purchases_per_range,
                                  "Average Purchase Price": avg_purchase_price_range,
                                  "Total Purchase Value": total_purchase_value_range,
                                  "Average Purchase Amount Per Person": avg_spend_per_age_range})

purchase_age_demo_clean = purchase_age_demo.style.format({'Average Purchase Price': "${:,.2f}", 
                                                          'Total Purchase Value': '${:,.2f}',
                                                          'Average Purchase Amount Per Person': '${:,.2f}'})

purchase_age_demo_clean

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Amount Per Person
Age Range,,,,
>10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,12,$3.04,$36.54,$3.32


Top Spenders
Run basic calculations to obtain the results in the table below
Create a summary data frame to hold the results
Sort the total purchase value column in descending order
Optional: give the displayed data cleaner formatting
Display a preview of the summary data frame

In [53]:
pd.options.display.float_format = '${:,.2f}'.format

#Top Spenders from the original data frame. 

#New DataFrame grouping on 'SN'
df_top_spenders = df.groupby('SN')

#Total Purchase Amount By the Top Spenders
total_purchases_tops = df_top_spenders['Purchase ID'].count()

#Average Purchase Price Spent By the Top Spenders
avg_purchase_price_tops = df_top_spenders['Price'].mean()

#Total Purchase Value Spent By the Top Spenders
total_spent_tops = df_top_spenders['Price'].sum()

#Summary DataFrame
top_spenders_new = pd.DataFrame({'Purchase Count': total_purchases_tops, 
                                 'Average Purchase Price': avg_purchase_price_tops,
                                 'Total Purchase Value': total_spent_tops})

#Summary DataFrame sorted by the total purchase value and showing a preview
top_spenders_new2 = top_spenders_new.sort_values(by='Total Purchase Value', ascending=False)

top_spenders_new2.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


Most Popular Items
Retrieve the Item ID, Item Name, and Item Price columns
Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value
Create a summary data frame to hold the results
Sort the purchase count column in descending order
Optional: give the displayed data cleaner formatting
Display a preview of the summary data frame

In [54]:
#Most popular items purchase information based on the original df. 
#Incomeplete. Needs Item Price

#New DF for the item information
item_df = df[['Item ID', 'Item Name', 'Price']]

#Grouped by Item Name and Item ID
item_df_grouped = item_df.groupby(['Item ID', 'Item Name'])

#Item Purchase Count Per Item
purchase_count = item_df_grouped.size()

#Item Price

#Total Purchase Value Per Item
purchase_value = item_df_grouped['Price'].sum()


#Summary DataFrame
item_info_start = pd.DataFrame({"Purchase Counts": purchase_count,
                                "Total Purchase Value": purchase_value})

popular_item_df = item_info_start.sort_values('Purchase Counts', ascending=False)
popular_item_df.head()

,,Purchase Counts,Total Purchase Value
Item ID,Item Name,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$50.76
145,Fiery Glass Crusader,9,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$31.77
82,Nirvana,9,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$8.16


Most Profitable Items
Sort the above table by total purchase value in descending order
Optional: give the displayed data cleaner formatting
Display a preview of the data frame

In [55]:
#Most profitable purchase information based on the original df

profitable_item_df = item_info_start.sort_values('Total Purchase Value', ascending=False)
profitable_item_df.head()

,,Purchase Counts,Total Purchase Value
Item ID,Item Name,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$50.76
82,Nirvana,9,$44.10
145,Fiery Glass Crusader,9,$41.22
92,Final Critic,8,$39.04
103,Singed Scalpel,8,$34.80
